# Partido para Quê?

As a Brazilian, I have many doubts about the necessity of many political parties in Brazil. Currently, there are 35 parties and an [Oxford study](https://www.bbc.com/portuguese/brasil-43288018) has shown that they could be reduce to only 2. Therefore, the objective of this project is answer the title question: "Partido para quê?" (which mean **"Parties for what?"** in Portuguese), throughout the analysis of voting patterns.

![Brazilian PArlamentar Distribution](https://i.imgur.com/5k2Y0Dd.png "Wikipedia 2013")


It will be used **Clustering Techniques** to classify Brazilians Senators based on their voting history of the past 8 years (last mandate).

In [1]:
# Install Requirements
!pip install pandas sqlalchemy numpy

# Data

## Description

The data is available in the file `database.db`. This database contains 2 tables:
- **Senadores**: Table with the Senators info (id, code, name, sex, state, party).
- **Votos**: Table with the voting info (id, senator, session, vote)

All the data used was collected though the [Senate API](http://legis.senado.leg.br/dadosabertos/docs/ui/index.html). The script with the source code which collects the data is available in [this file](https://github.com/Fernandohf/Partido-para-que/blob/master/update_db.py).

## Visualization

Lets start by loading the data from the database to two **Pandas** `DataFrame` objects.

In [2]:
# Imports
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# Create DB Engine and Session
engine = create_engine('sqlite:///database.db')
DBSession = sessionmaker(bind=engine)
session = DBSession()

# All field from Senatores Table
query_senators = 'SELECT * FROM Senadores'
senators = pd.read_sql(query_senators, session.bind)

# 5 First values
senators.head()


,SenadorID,SenadorCod,NomeCompleto,Sexo,Estado,PartidoSigla
0,1,4981,Acir Marcos Gurgacz,Masculino,RO,PDT
1,2,5140,Airton Sandoval Santana,Masculino,SP,MDB
2,3,945,Alvaro Fernandes Dias,Masculino,PR,PODE
3,4,4988,Ana Amélia de Lemos,Feminino,RS,PP
4,5,5529,Antonio Augusto Junho Anastasia,Masculino,MG,PSDB


Similarly, we can do the same to the voting table.

In [3]:

# All field from Votos Table
query_votes = 'SELECT * FROM Votos'
votes = pd.read_sql(query_votes, session.bind)

# Close DB session
session.close()

# Show first 5 values
votes.head()

,VotoID,SenadorID,SessaoID,Voto
0,0,40,44809,Sim
1,3,40,44949,Sim
2,7,40,45483,Sim
3,8,40,44589,Sim
4,9,40,42415,Sim


## Cleaning and Preparation

Let's if the DataFrames have any missing values.

In [4]:
# Missing values check
print("Total of missing entries in Senators: " + str(senators.isnull().sum().sum()))
print("Total of missing entries in Votes: " + str(votes.isnull().sum().sum()))

Total of missing entries in Senators: 0
Total of missing entries in Votes: 0


Apparently, there are no missing values in the table. 

Checking the ~~bad~~ [API Documentation](http://legis.senado.leg.br/dadosabertos/docs/ui/index.html), we can see that there are multiple possible values for the `vote` column. With the exception of `Sim` and `Não`, all values represent different kinds of abstesions. 

**These values are listed bellow.**


In [5]:
# All possible votes values
votes['Voto'].unique()

array(['Sim', 'MIS', 'AP ', 'P-NRV', 'Não', 'LS ', 'Abstenção',
       'Presidente (art. 51 RISF)', 'LP ', 'NCom', 'Obstrução', 'NA ',
       'P-OD', 'PSF', 'Votou', 'LAP'], dtype=object)

These values will be remaped to the following values.

Vote | Value
-------|-------
Sim | 1
Não | 0
All others (abstensions) | 0.5

For this, the following function was defined and applied.

In [6]:
def votes_remap(v):
    # Check if columns is numeric
    if isinstance(v, str):
        if v == 'Sim':
            return 1
        elif v == 'Não':
            return 0
        else:
            return .5
    else:
        return v
votes['Voto'] = votes['Voto'].apply(votes_remap)
votes['Voto'] = votes['Voto'].astype(float)
votes.head()


,VotoID,SenadorID,SessaoID,Voto
0,0,40,44809,1.0
1,3,40,44949,1.0
2,7,40,45483,1.0
3,8,40,44589,1.0
4,9,40,42415,1.0


Alright, now we have two clean dataframes containig the information needed to perform the analysis. However, the data is still defined in **two different DataFrames**. We need to add the columns `NomeCompleto` and `PartidoSigla` to a **final DataFrame**.

Using pandas, It is simple to combine the DataFrames together.

In [8]:
# Senators info
senators_info = senators[['SenadorCod', 'NomeCompleto', 'PartidoSigla']]
# Match columns name
senators_info.columns = ['SenadorID', 'NomeCompleto', 'PartidoSigla']

# Combine dataframes
combined = pd.merge(votes, senators_info, how='left', on='SenadorID')

# Check for inconsitensy
print("Null values of combined DataFrame: " + str(combined.isnull().sum().sum()))

combined.head()

Null values of combined DataFrame: 0


,VotoID,SenadorID,SessaoID,Voto,NomeCompleto,PartidoSigla
0,0,40,44809,1.0,José Agripino Maia,DEM
1,3,40,44949,1.0,José Agripino Maia,DEM
2,7,40,45483,1.0,José Agripino Maia,DEM
3,8,40,44589,1.0,José Agripino Maia,DEM
4,9,40,42415,1.0,José Agripino Maia,DEM


In [9]:
# Average senators per session
combined['SessaoID'].value_counts().mean()

74.55339805825243

Finally, let's perform a **Pivot Table** on the `combined` DataFrame. This way, the resulting DataFrame will correspond to a two-dimensional data structure. In other words, each row will be a senator and the columns will represent the voting behaviour of this senator for each session.

In [15]:
# Selected columns from combined
cols = ['SessaoID', 'Voto', 'NomeCompleto', 'PartidoSigla']
combined_reduced = combined[cols]
pivot = pd.pivot_table(combined_reduced, values='Voto', index=['NomeCompleto', 'PartidoSigla'], columns=['SessaoID'])
print('Total NaN entries: ' + str(pivot.isnull().sum().sum()))
print('Final DataFrame shape: ' + str(pivot.shape))
pivot.index.names = ['Name', 'Party']
pivot.head()


Total NaN entries: 664
Final DataFrame shape: (81, 103)


,SessaoID,22358,22362,22376,22377,22391,22403,22408,22422,22424,22443,...,68626,69058,69537,70111,71706,74732,76256,76768,77207,78620
Name,Party,,,,,,,,,,,,,,,,,,,,,
Acir Marcos Gurgacz,PDT,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.5,0.5,1.0,...,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0
Airton Sandoval Santana,MDB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
Alvaro Fernandes Dias,PODE,0.5,0.5,0.0,1.0,0.0,0.5,0.5,0.5,0.5,1.0,...,1.0,0.5,0.5,0.5,1.0,0.5,0.5,0.5,0.5,0.5
Ana Amélia de Lemos,PP,1.0,0.5,0.5,1.0,0.0,1.0,0.5,0.0,1.0,0.5,...,0.5,1.0,1.0,0.5,1.0,1.0,1.0,1.0,0.5,0.0
Antonio Augusto Junho Anastasia,PSDB,1.0,1.0,1.0,1.0,0.0,0.0,0.5,0.0,1.0,1.0,...,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,1.0,0.5


In [16]:
# Rows with missing values
pivot.isnull().sum(axis=1).astype(bool).sum()

27

After this pivot, there are missing values due to new senators (every 4 years) and substitutions during the mandates. In order to clean this final dataframe, these rules will be followed:
 - **Remove rows** with at least 80 non-NaN values.
 - **Replace NaN** with random value (0, 1 , 0.5).
 
 Additionally, we will add the party column.

In [17]:
# Drop rows with at least 10 missing values
dropped = pivot.dropna(axis=0, thresh=80)
print(dropped.shape)

# Fill remaining NaN
np.random.seed(42)
final = dropped.fillna(np.random.choice([0., .5, 1.]))
# Check for inconsitensy
print("Null values of Final DataFrame: " + str(final.isnull().sum().sum()))
final.head()


(72, 103)
Null values of Final DataFrame: 0


,SessaoID,22358,22362,22376,22377,22391,22403,22408,22422,22424,22443,...,68626,69058,69537,70111,71706,74732,76256,76768,77207,78620
Name,Party,,,,,,,,,,,,,,,,,,,,,
Acir Marcos Gurgacz,PDT,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.5,0.5,1.0,...,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0
Alvaro Fernandes Dias,PODE,0.5,0.5,0.0,1.0,0.0,0.5,0.5,0.5,0.5,1.0,...,1.0,0.5,0.5,0.5,1.0,0.5,0.5,0.5,0.5,0.5
Ana Amélia de Lemos,PP,1.0,0.5,0.5,1.0,0.0,1.0,0.5,0.0,1.0,0.5,...,0.5,1.0,1.0,0.5,1.0,1.0,1.0,1.0,0.5,0.0
Antonio Augusto Junho Anastasia,PSDB,1.0,1.0,1.0,1.0,0.0,0.0,0.5,0.0,1.0,1.0,...,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,1.0,0.5
Antonio Carlos Valadares,PSB,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.5,1.0,0.5,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0


At last, this is the final format of the **DataFrame** which will be used in the forthcoming analysis.

# Clustering
The clustering technique consists of grouping similar senators based in their voting behaviour. We will use **Scikit-Learn's** built-in clustering classses to perfom multiple analisys. 

First, let's see if the senator follow a party rule in their voting pattern. We can see that there are a total of 19 parties, then this will be the number of clusters we will be searching for.





In [18]:
print("Parties: ")
print(senators['PartidoSigla'].unique())
print(len(senators['PartidoSigla'].unique()))

Parties: 
['PDT' 'MDB' 'PODE' 'PP' 'PSDB' 'PSB' 'PTB' 'PR' 'PPS' 'DEM' 'PRB' 'PTC'
 'PT' 'DC' 'PROS' 'PSD' 'REDE' 'S/Partido' 'PCdoB']
19


In [25]:
# imports
from sklearn.cluster import KMeans

kmeans_19 = KMeans(n_clusters=19, random_state=42)
distances_19 = kmeans_19.fit_transform(final)

# Crosstable
labels_19 = kmeans_19.labels_
parties = final.index.get_level_values(1).to_series()
cross_tab_19 = pd.crosstab(labels_19, parties)
cross_tab_19


Party,DEM,MDB,PCdoB,PDT,PODE,PP,PPS,PR,PRB,PROS,PSB,PSD,PSDB,PT,PTB,PTC,REDE,S/Partido
row_0,,,,,,,,,,,,,,,,,,
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0
2,0,4,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,2,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,1,0
6,1,4,0,0,0,1,0,2,0,0,1,0,1,0,0,0,0,0
7,0,3,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0
8,2,1,0,0,0,2,0,0,0,0,0,0,7,0,0,0,0,0


## Parties Similarities / Differences

In the **Cross Table** above we can see that even though som eparties were pretty clustered, others were really similiar to others. 

Examples:

- Senators from PT were assigned to groups 1, 5, 9, and 11.
- Meanwhile the majority of senator from PSDB were assigned to group 8.


In [29]:
# Situation / Oposition
kmeans_2 = KMeans(n_clusters=2, random_state=42)
ditances_2 = kmeans_2.fit_transform(final)

# Crosstable
labels_2 = kmeans_2.labels_
cross_tab_2 = pd.crosstab(labels_2, parties)
cross_tab_2

Party,DEM,MDB,PCdoB,PDT,PODE,PP,PPS,PR,PRB,PROS,PSB,PSD,PSDB,PT,PTB,PTC,REDE,S/Partido
row_0,,,,,,,,,,,,,,,,,,
0,0,1,1,1,0,0,0,0,0,0,2,0,0,9,0,0,1,1
1,4,16,0,1,4,4,1,3,1,1,2,5,11,0,2,1,0,0


Here, we can see a trend between current government and oposition.

The following party behaviour is shown by the clustering:

Oposition | Situation|Independent
:---|:----:|---:
PT, REDE, PCdoB | MDB, PODE, PP, PSDB, PTB, PR, PPS, DEM, PRB, PTC, PROS, PSD | PDT, PSB, S/Partido 